In [27]:
import datasets

items_data = datasets.load_dataset("ed-donner/items_lite")

In [8]:
train_dataset = items_data["train"]
validation_dataset = items_data["validation"]   
test_dataset = items_data["test"]




In [14]:
df_train = train_dataset.to_pandas()
df_test = test_dataset.to_pandas()
df_validation = validation_dataset.to_pandas()

In [ ]:

from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = HashingVectorizer(n_features=3000, stop_words="english", binary=True)

X_train = vectorizer.fit_transform(df_train["summary"])

X_test = vectorizer.transform(df_test["summary"])

Y_train = df_train["price"]

Y_test = df_test["price"]
Y_train.iloc[0].dtype


dtype('float64')

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()


In [28]:
model.fit(X_train, Y_train)

,"fit_intercept fit_intercept: bool, default=TrueWhether to calculate the intercept for this model. If setto False, no intercept will be used in calculations(i.e. data is expected to be centered).",True
,"copy_X copy_X: bool, default=TrueIf True, X will be copied; else, it may be overwritten.",True
,"tol tol: float, default=1e-6The precision of the solution (`coef_`) is determined by `tol` whichspecifies a different convergence criterion for the `lsqr` solver.`tol` is set as `atol` and `btol` of :func:`scipy.sparse.linalg.lsqr` whenfitting on sparse training data. This parameter has no effect when fittingon dense data... versionadded:: 1.7",1e-06
,"n_jobs n_jobs: int, default=NoneThe number of jobs to use for the computation. This will only providespeedup in case of sufficiently large problems, that is if firstly`n_targets > 1` and secondly `X` is sparse or if `positive` is setto `True`. ``None`` means 1 unless in a:obj:`joblib.parallel_backend` context. ``-1`` means using allprocessors. See :term:`Glossary ` for more details.",None
,"positive positive: bool, default=FalseWhen set to ``True``, forces the coefficients to be positive. Thisoption is only supported for dense arrays.For a comparison between a linear regression model with positive constraintson the regression coefficients and a linear regression without such constraints,see :ref:`sphx_glr_auto_examples_linear_model_plot_nnls.py`... versionadded:: 0.24",False


In [ ]:
import numpy as np

predicted_y = [max(p,0) for p in model.predict(X_test)]

In [36]:
import plotly.express as px

px.scatter(x=predicted_y, y=Y_test, title="Predicted vs Actual", labels={"x": "Predicted", "y": "Actual"})

In [39]:
from sklearn.metrics import mean_squared_error,mean_absolute_error

print(mean_squared_error(predicted_y, Y_test))
print(mean_absolute_error(predicted_y, Y_test))



17814.559340938893
94.21966227031874


In [46]:
Y_random = np.random.randint(0, 1000, size=X_test.shape[0])

In [48]:
print(mean_squared_error(Y_random, Y_test))
print(mean_absolute_error(Y_random, Y_test))

243396.6679799
414.39077000000003


In [49]:
px.scatter(x=Y_random, y=Y_test, title="Predicted vs Actual", labels={"x": "Predicted", "y": "Actual"})

In [51]:

from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=3000, stop_words="english")

X_train = vectorizer.fit_transform(df_train["summary"])

X_test = vectorizer.transform(df_test["summary"])

Y_train = df_train["price"]

Y_test = df_test["price"]
Y_train.iloc[0].dtype


dtype('float64')

In [ ]:
MODEL_NAME = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
bert = AutoModel.from_pretrained(MODEL_NAME)

In [1]:
import torch.nn as nn
class PriceNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 3000),
            nn.ReLU(),
            nn.Linear(3000,2000),
            nn.ReLU(),
            nn.Linear(2000, 2000),
            nn.ReLU(),
            nn.Linear(2000, 1),
            nn.ReLU()
        ).to(device)
    
    def forward(self, x):
        return self.net(x)

In [4]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PriceNN(5000).to(device)
model.load_state_dict(torch.load("C:\\Users\\srini\\my-python-projects\\llm_engineering_Jan_2026\\week6\\pricer\\nn_price_model\\nn_price_model.pth"))
model.eval()


PriceNN(
  (net): Sequential(
    (0): Linear(in_features=5000, out_features=3000, bias=True)
    (1): ReLU()
    (2): Linear(in_features=3000, out_features=2000, bias=True)
    (3): ReLU()
    (4): Linear(in_features=2000, out_features=2000, bias=True)
    (5): ReLU()
    (6): Linear(in_features=2000, out_features=1, bias=True)
    (7): ReLU()
  )
)

In [12]:
from datasets.arrow_dataset import Dataset
import datasets

dataset = datasets.load_dataset("ed-donner/items_lite")

In [18]:
dataset_test = dataset["test"]

In [19]:
X_test = dataset_test.to_pandas()["summary"]
y_test = dataset_test.to_pandas()["price"]


In [20]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer


# vectorizer = TfidfVectorizer(max_features=5000, stop_words="english")
import joblib
vectorizer = joblib.load(r'C:\Users\srini\my-python-projects\llm_engineering_Jan_2026\week6\pricer\nn_price_model\hashing_vectorizer.joblib')

In [21]:
X_test = vectorizer.transform(X_test)


In [23]:

class PriceDataSet(Dataset):
    def __init__(self, X_sparse, y):
        self.X = X_sparse
        self.y = y
    
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, idx):
        return torch.tensor(self.X[idx].toarray().flatten(), dtype=torch.float32).to(device), torch.tensor(self.y[idx], dtype=torch.float32).to(device)


In [24]:

from torch.utils.data import DataLoader
test_data = PriceDataSet(X_test, y_test)
test_loader = DataLoader(test_data, batch_size=500, shuffle=False)

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from torch.utils.data import Dataset

def predict_price(model, X):
    model.eval()
    X_tensor = torch.tensor(X.toarray(), dtype=torch.float32).to(device)
    with torch.no_grad():
        y_pred = model(X_tensor)
    return y_pred

y_pred = predict_price(model, X_test).flatten().cpu().numpy()

In [25]:
import plotly.express as px

px.scatter(x=y_pred, y=y_test, title="Predicted vs Actual", labels={"x": "Predicted", "y": "Actual"})

In [26]:
from sklearn.metrics import mean_squared_error,mean_absolute_error

print(mean_squared_error(y_pred, y_test))
print(mean_absolute_error(y_pred, y_test))

8188.521095187398
51.44132956928254
